# Глава 29. Перегрузка операторов

## Основы

**Перегрузка операторов** - перехватывание встроенных операций с помощью методов классов - интерпретатор автоматически вызывает эти методы при выполнении встроенных операций над экземплярами классов, а методы должны возвращать значения, которые будут интерпретироваться как результаты соответствующих операций.

* Перегрузка операторов в  языке Python позволяет классам участвовать в обычных операциях.


* Классы в языке Python могут перегружать все операторы выражений.


* Классы могут также перегружать такие операции, как вывод, вызов функций, обращение к атрибутам и так далее.


* Перегрузка делает экземпляры классов более похожими на встроенные типы.


* Перегрузка заключается в реализации в классах методов со специальными именами.

## Общие методы перегрузки операторов

| Метод | Перегружает | Вызывается |
| --- | --- | --- |
| `__init__` | Конструктор | При создании объекта: `X = Class(args)` |
| `__del__` | Деструктор | При уничтожении объекта |
| `__add__` | Оператор `+` | `X + Y`, `X += Y`, если отсутствует метод `__iadd__` |
| `__or__` | Оператор `｜` (побитовое ИЛИ) | `X ｜ Y`, `X ｜= Y`, если отсутствует метод `__ior__` |
| `__repr__`, `__str__` | Вывод, преобразование | `print(X)`, `repr(X)`, `str(X)` |
| `__call__` | Вызовы функции | `X(*args, **kargs)` |
| `__getattr__` | Обращение к атрибуту | `X.undefined` |
| `__setattr__` | Присваивание атрибуту | `X.any = value` |
| `__delattr__` | Удаление атрибута | `del X.any` |
| `__getattribute__` | Обращение к атрибуту | `X.any` |
| `__getitem__` | Доступ к элементу по индексу, извлечение среза, итерации | `X[key]`, `X[i:j]`, циклы `for` и другие конструкции итерации, при отсутствии метода `__iter__` |
| `__setitem__` | Присваивание элементу по индексу или срезу | `X[key] = value`, `X[i:j] = sequence` |
| `__delitem__` | Удаление элемента по индексу или среза | `del X[key]`, `del X[i:j]` |
| `__len__` | Длина | `len(X)`, проверка истинности, если отсутствует метод `__bool__` |
| `__bool__` | Проверка логического значения | `bool(X)` , проверка истинности |
| `__lt__, __gt__, __le__, __ge__, __eq__, __ne__` | Сравнивание | `X < Y`, `X > Y`, `X <= Y`, `X >= Y`, `X == Y`, `X != Y` |
| `__radd__` | Правосторонний оператор `+` | Не_экземпляр + `X` |
| `__iadd__` | Добавление (увеличение) | `X += Y` (в ином случае `__add__`) |
| `__iter__, __next__` | Итерационный контекст | `I=iter(X)`, `next(I)`; циклы `for`, оператор `in` (если не определен метод `__contains__`), все типы генераторов, `map(F, X)` и другие |
| `__contains__` | Проверка на вхождение | `item in X` (где `X` – любой итерируемый объект) |
| `__index__` | Целое число | `hex(X)`, `bin(X)` , `oct(X)` , `O[X]` , `O[X:]` |
| `__enter__, __exit__` | Менеджеры контекстов (глава 33) | `with obj as var:` |
| `__get__, __set__, __delete__` | Дескрипторы атрибутов (глава 37) | `X.attr`, `X.attr = value`, `del X.attr` |
| `__new__` | Создание (глава 39) | Вызывается при создании объектов, перед вызовом метода `__init__` |

Методы перегрузки операторов могут наследоваться от суперклассов, если они отсутствуют в самом классе, как и любые другие методы.

Методы перегрузки являются необязательными - если какой-то метод не реализован, это лишь означает, что соответствующая ему операция не поддерживается классом, а при попытке применить такую операцию возбуждается исключение.

Некоторые встроенные операции, такие как вывод, имеют реализацию по умолчанию (в Python 3.0 они наследуются от класса `object`, являющегося суперклассом для всех объектов), но большинство операций будут вызывать исключение, если класс не предусматривает реализацию соответствующего метода.

In [1]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## Доступ к элементам по индексу и извлечение срезов: `__getitem__` и `__setitem__`

Если метод **`__getitem__`** присутствует в  определении класса (или наследуется им), он автоматически будет вызываться интерпретатором в  случае **применения операций индексирования к экземплярам**.

Например, следующий класс возвращает квадрат значения индекса:

In [2]:
class Indexer:
    def __getitem__(self, index):
        return index ** 2

In [3]:
X = Indexer()
X[2]  # Выражение X[i] вызывает X.__getitem__(i)

4

In [4]:
for i in range(5):
    print(X[i], end=' ')

0 1 4 9 16 

### Извлечение срезов

`__getitem__` вызывается не только при выполнении операции обращения к элементу по индексу, но **и при извлечении срезов**.

Параметры среза определяются с помощью **объекта среза**, который и передается реализации операции индексирования списка.

In [5]:
L = list(range(10))

L[slice(2, 4)] # Извлечение среза с помощью объекта среза

[2, 3]

In [6]:
L[slice(None, None, 2)]

[0, 2, 4, 6, 8]

Пример класса, поддерживающего извлечение среза:

In [7]:
class Indexer:
    data = list(range(10))
    def __getitem__(self, index):  # вызывается при индексировании или
        print('getitem:', index)   # извлечении среза
        return self.data[index]    # выполняет индексирование или
                                   # извлекает срез

In [8]:
X = Indexer()

In [9]:
X[0]

getitem: 0


0

In [10]:
X[-1]

getitem: -1


9

Когда метод вызывается для извлечения среза, он получает объект среза, который просто передается списку, встроенному в класс `Indexer`, в виде выражения обращения по индексу:

In [11]:
X[2:4]

getitem: slice(2, 4, None)


[2, 3]

Метод **`__setitem__`** присваивания элементу по индексу точно так же обслуживает обе операции – **присваивания элементу по индексу и присваивание срезу**.

В последнем случае он получает объект среза, который может передаваться
другим операциям присваивания по индексу:

```
def __setitem__(self, index, value):
    ...
    self.data[index] = value
```

## Итерации по индексам: `__getitem__`

Инструкция `for` многократно применяет операцию индексирования к последовательности, используя индексы от нуля и выше, пока не будет получено исключение выхода за границы.

Благодаря этому метод `__getitem__` представляет собой один из способов перегрузки итераций - если этот метод реализован, инструкции циклов `for` будут вызывать его на каждом шаге цикла, с постоянно увеличивающимся смещением.

In [12]:
class stepper:
    def __getitem__(self, i):
        return self.data[i]

In [13]:
X = stepper()
X.data = 'Spam'
X[1]

'p'

In [14]:
for item in X:
    print(item, end=' ')

S p a m 

Оператор проверки на принадлежность `in`, генераторы списков, встроенная функция `map`, присваивание списков и кортежей и конструкторы типов также автоматически вызывают метод `__getitem__`,
если он определен:

In [15]:
'p' in X

True

In [16]:
list(map(str.upper, X))

['S', 'P', 'A', 'M']

In [17]:
(a, b, c, d) = X
a, c, d

('S', 'a', 'm')

In [18]:
X

## Итераторы: `__iter__` и `__next__`

Прием, основанный на использовании метода `__getitem__`, представленный в предыдущем разделе, действительно работает, однако он используется операциями, выполняющими итерации, в  самом крайнем случае.

В  настоящее время **все итерационные контексты в  языке Python пытаются сначала использовать метод `__iter__`**, и только потом – метод `__getitem__`.

С технической точки зрения итерационные контексты вызывают встроенную функцию `iter`, чтобы определить наличие метода `__iter__`, который должен возвращать объект итератора.

Если он предоставляется, то интерпретатор Python будет вызывать метод `__next__` объекта итератора для получения элементов до тех пор, пока не будет возбуждено исключение `StopIteration`.

Если метод `__iter__` отсутствует, интерпретатор переходит на использование схемы с применением метода `__getitem__` и начинает извлекать элементы по индексам, пока не будет возбуждено исключение `IndexError`.

Кроме того, для удобства предоставляется **встроенная функция `next`**, позволяющая выполнять итерации вручную: вызов `next(I)` – это то же самое, что вызов `I.__next__()`.

### Итераторы, определяемые пользователями

Следующий класс определяет итератор, который возвращает квадраты чисел:

In [19]:
class Squares:
    def __init__(self, start, stop):  # сохранить сост-ие при создании
        self.value = start - 1
        self.stop = stop
    
    def __iter__(self):               # возвращает итератор в iter()
        return self
    
    def __next__(self):               # возвр-т квадрат в каждой ит-ии
        if self.value == self.stop:   # также вызывается ф-ей next
            raise StopIteration
        self.value += 1
        return self.value ** 2

In [20]:
for i in Squares(1, 5):  # for вызывает iter(), кот. выз. __iter__()
    print(i, end=' ')    # на каждой итерации выз-тся __next__()

1 4 9 16 25 

In [21]:
list(Squares(5, 9))

[25, 36, 49, 64, 81]

Здесь объект итератора  – это просто экземпляр `self`, поэтому метод `__next__` является частью этого класса. **В более сложных ситуациях объект итератора может быть определен как отдельный класс и объект** со своей собственной информацией о состоянии, с целью поддержки нескольких активных итераций на одних и  тех же данных.

Итераторы, реализованные на основе метода `__iter__`, иногда могут оказаться более сложными и менее удобными, чем метод `__getitem__`. Но они действительно предназначены для итераций, а  не для случайной индексации,  – фактически они вообще не перегружают операцию индексирования:

In [22]:
X = Squares(1, 5)
X[3]

TypeError: 'Squares' object does not support indexing

В отличие от `__getitem__`, схема на основе метода `__iter__` предназначена для выполнения обхода элементов один раз, а не несколько. 

Например, элементы класса `Squares` можно обойти всего один раз – для каждой последующей итерации необходимо будет создавать новый объект итератора:

In [23]:
X = Squares(1, 5)
[n for n in X]  # Получить все элементы

[1, 4, 9, 16, 25]

In [24]:
[n for n in X] # Теперь объект пуст

[]

In [25]:
[n for n in Squares(1, 5)] # Создать новый объект итератора

[1, 4, 9, 16, 25]

### Несколько итераторов в одном объекте

Пример при выполнении обхода элементов встроенных типов, таких как строка:

In [26]:
S = 'ace'
for x in S:
    for y in S:
        print(x + y, end=' ')

aa ac ae ca cc ce ea ec ee 

Здесь внешний цикл получает итератор строки вызовом функции `iter` и каждый вложенный цикл делает то же самое, чтобы получить независимый итератор. Т.к. каждый итератор хранит свою собственную информацию о состоянии, каждый цикл управляет своим собственным положением в строке, независимо от любых других активных циклов.

>Чтобы обеспечить поддержку множества независимых итераторов, метод `__iter__` должен не просто возвращать аргумент `self`, а создавать новый объект итератора со своей информацией о состоянии.

В следующем примере определяется класс итератора, который пропускает каждый второй элемент. Поскольку объект итератора создается заново для каждой итерации, он обеспечивает поддержку нескольких активных циклов одновременно:

In [27]:
class SkipIterator:
    def __init__(self, wrapped):
        self.wrapped = wrapped  # информация о состоянии
        self.offset = 0
        
    def __next__(self):
        if self.offset >= len(self.wrapped): # завершить итерации
            raise StopIteration
        else:
            item = self.wrapped[self.offset] # иначе перешагнуть и вернуть
            self.offset += 2
            return item

        
class SkipObject:
    def __init__(self, wrapped):
        self.wrapped = wrapped  # сохранить используемый элемент
        
    def __iter__(self):
        return SkipIterator(self.wrapped) # каждый раз новый итератор

In [28]:
alpha = 'abcdef'
skipper = SkipObject(alpha)       # создать объект-контейнер
I = iter(skipper)                 # создать итератор для него
print(next(I), next(I), next(I))  # обойти элементы 0, 2, 4

a c e


In [29]:
for x in skipper:     # for вызывает __iter__ автоматически
    for y in skipper: # вложенные циклы for также вызывают __iter__
        print(x + y, end=' ')  # каждый итератор помнит свое состояние,
                               # смещение

aa ac ae ca cc ce ea ec ee 

## Проверка на вхождение: `__contains__`, `__iter__` и `__getitem__`

Перегрузка операторов нередко образует **многослойную архитектуру**: классы могут представлять реализацию специфических методов или обобщенные альтернативы, используемые в крайнем случае, например:

Операция проверки логического значения сначала пытается вызвать специализированный метод `__bool__`, а в случае его отсутствия вызывает более обобщенный метод `__len__`.

Метод **`__contains__`** обеспечивает специализированную поддержку операции проверки на членство - этот метод имеет преимущество перед методом `__iter__`, который в свою очередь пользуется преимуществом перед `__getitem__`. В случае **отображений** метод `__contains__` должен определять членство, применяя ключи (и может использовать быструю операцию поиска), а в случае **последовательностей** - производить поиск.

In [30]:
class Iters:
    def __init__(self, value):
        self.data = value
    
    def __getitem__(self, i):         # Крайний случай для итераций
        print('get[%s]:' % i, end='') # А также для индексирования и срезов
        return self.data[i]
    
    def __iter__(self):               # Предпочтительный для итераций
        print('iter=> ', end='')      # Возможен только 1 активный итератор
        self.ix = 0
        return self
    
    def __next__(self):
        print('next:', end='')
        if self.ix == len(self.data):
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    
    def __contains__(self, x):        # Предпочт-ый для оператора ‘in’
        print('contains: ', end='')
        return x in self.data

In [31]:
X = Iters([1, 2, 3, 4, 5])  # Создать экземпляр
print(3 in X)               # Проверка на вхождение
for i in X:                 # Циклы
    print(i, end=' | ') 
print('\n')
print([i ** 2 for i in X])  # Другие итерационные контексты
print( list(map(bin, X)) )

I = iter(X)                 # Обход вручную (именно так действуют
while True:                 # другие итерационные контексты)
    try:
        print(next(I), end=' @ ')
    except StopIteration:
        break

contains: True
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next:

iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1 @ next:2 @ next:3 @ next:4 @ next:5 @ next:

Eсли не использовать `__contains__`, то операция проверки на вхождение будет использовать обобщенный метод `__iter__`

In [32]:
class Iters:
    def __init__(self, value):
        self.data = value
    
    def __getitem__(self, i):         # Крайний случай для итераций
        print('get[%s]:' % i, end='') # А также для индексирования и срезов
        return self.data[i]
    
    def __iter__(self):               # Предпочтительный для итераций
        print('iter=> ', end='')      # Возможен только 1 активный итератор
        self.ix = 0
        return self
    
    def __next__(self):
        print('next:', end='')
        if self.ix == len(self.data):
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item
    
#     def __contains__(self, x):        # Предпочт-ый для оператора ‘in’
#         print('contains: ', end='')
#         return x in self.data

In [33]:
X = Iters([1, 2, 3, 4, 5])  # Создать экземпляр
print(3 in X)               # Проверка на вхождение
for i in X:                 # Циклы
    print(i, end=' | ')
print('\n')
print([i ** 2 for i in X])  # Другие итерационные контексты
print( list(map(bin, X)) )
I = iter(X)                 # Обход вручную (именно так действуют
while True:                 # другие итерационные контексты)
    try:
        print(next(I), end=' @ ')
    except StopIteration:
        break

iter=> next:next:next:True
iter=> next:1 | next:2 | next:3 | next:4 | next:5 | next:

iter=> next:next:next:next:next:next:[1, 4, 9, 16, 25]
iter=> next:next:next:next:next:next:['0b1', '0b10', '0b11', '0b100', '0b101']
iter=> next:1 @ next:2 @ next:3 @ next:4 @ next:5 @ next:

И, наконец, если и `__contains__`, и `__iter__` не используются, то при проверке на вхождение и в других итерационных методах используется метод `__getitem__`, которому последовательно передаются индексы в порядке возрастания:

In [34]:
class Iters:
    def __init__(self, value):
        self.data = value
    
    def __getitem__(self, i):         # Крайний случай для итераций
        print('get[%s]:' % i, end='') # А также для индексирования и срезов
        return self.data[i]
    
    def __next__(self):
        print('next:', end='')
        if self.ix == len(self.data):
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item

In [35]:
X = Iters([1, 2, 3, 4, 5])  # Создать экземпляр
print(3 in X)               # Проверка на вхождение
for i in X:                 # Циклы
    print(i, end=' | ')
print('\n')
print([i ** 2 for i in X])  # Другие итерационные контексты
print( list(map(bin, X)) )
I = iter(X)                 # Обход вручную (именно так действуют
while True:                 # другие итерационные контексты)
    try:
        print(next(I), end=' @ ')
    except StopIteration:
        break

get[0]:get[1]:get[2]:True
get[0]:1 | get[1]:2 | get[2]:3 | get[3]:4 | get[4]:5 | get[5]:

get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:[1, 4, 9, 16, 25]
get[0]:get[1]:get[2]:get[3]:get[4]:get[5]:['0b1', '0b10', '0b11', '0b100', '0b101']
get[0]:1 @ get[1]:2 @ get[2]:3 @ get[3]:4 @ get[4]:5 @ get[5]:

## Обращения к атрибутам: `__getattr__` и `__setattr__`

Метод **`__getattr__` выполняет операцию получения ссылки на атрибут**. 

Если говорить более определенно, он вызывается с именем атрибута в виде строки всякий раз, когда обнаруживается попытка получить ссылку на неопределенный (несуществующий) атрибут. **Этот метод не вызывается, если интерпретатор может обнаружить атрибут посредством выполнения процедуры поиска в дереве наследования**. Вследствие этого метод `__getattr__` удобно использовать для обобщенной обработки запросов к атрибутам.

In [36]:
class empty:
    def __getattr__(self, attrname):
        if attrname == 'age':
            return 40
        else:
            raise AttributeError(attrname)
            
X = empty()
X.age

40

In [37]:
X.name

AttributeError: name

В этом примере класс `empty` и  его экземпляр `X` не имеют своих собственных атрибутов, поэтому при обращении к атрибуту `X.age` вызывается метод `__getattr__` – в аргументе `self` передается экземпляр (`X`), а в аргументе `attrname` – строка с именем неопределенного атрибута (`age`).

Класс выглядит так, как если бы он действительно имел атрибут `age`, возвращая результат обращения к имени `X.age` (`40`). В результате получается **атрибут, вычисляемый динамически**.

Метод перегрузки **`__setattr__` перехватывает все попытки присваивания значений атрибутам**.

Если этот метод определен, выражение `self.attr = value` будет преобразовано в вызов метода `self.__setattr_('attr', value)`.

In [38]:
class accesscontrol:
    def __setattr__(self, attr, value):
        if attr == 'age':
            self.__dict__[attr] = value
        else:
            raise AttributeError(attr + ' not allowed')
            
X = accesscontrol()
X.age = 40
X.age

40

In [39]:
X.name = 'mel'

AttributeError: name not allowed

### Другие способы управления атрибутами

* **Метод `__getattribute__`** вызывается при обращениях к любым атрибутам, не только к  неизвестным; но при реализации этого метода следует быть еще более осторожным, чем при реализации метода `__getattr__`, чтобы избежать зацикливания.


* **Встроенная функция `property`** позволяет ассоциировать специальные методы с операциями чтения и записи над определенными атрибутами класса.


* Дескрипторы предоставляют возможность ассоциировать методы `__get__` и `__set__` класса с операциями доступа к определенным атрибутам класса.

### Имитация частных атрибутов экземпляра: часть 1

In [40]:
class PrivateExc(Exception):  # Подробнее об исключениях позднее
    pass

class Privacy:
    def __setattr__(self, attrname, value): # Выз-ся self.attrname = value
        if attrname in self.privates:
            raise PrivateExc(attrname, self)
        else:
            self.__dict__[attrname] = value # Self.attrname = value
                                            # вызовет зацикливание!
class Test1(Privacy):
    privates = ['age']
    
class Test2(Privacy):
    privates = ['name', 'pay']
    def __init__(self):
        self.__dict__['name'] = 'Tom'

In [41]:
x = Test1()
y = Test2()

In [42]:
x.name = 'Bob'

In [43]:
y.name = 'Sue'

PrivateExc: ('name', <__main__.Test2 object at 0x7f7f70988470>)

In [44]:
y.age = 30

In [45]:
x.age = 40

PrivateExc: ('age', <__main__.Test1 object at 0x7f7f70988198>)

In [46]:
class Test3(Privacy):
    privates = ['name', 'pay']
    def __init__(self):
        self.name = 'Tom'  # __setattr__ в суперклассе Privacy не даст создать ни одного экземлпяра Test3
                           # нужно как в Test2 использовать self.__dict__['name'] = 'Tom'

In [47]:
Test3()

PrivateExc: ('name', <__main__.Test3 object at 0x7f7f70988ba8>)

Фактически это лишь первая прикидочная реализация частных атрибутов в языке Python (то есть запрет на изменение атрибутов вне класса). 

Несмотря на то что язык Python не поддерживает возможность объявления частных атрибутов, такие приемы, как этот, могут их имитировать. Однако, хотя таким способом можно имитировать сокрытие атрибутов, тем не менее он почти никогда не используется на практике.

## Строковое представление объектов: `__repr__` и `__str__`

Если метод `__repr__` (или родственный ему метод `__str__`) определен, он автоматически будет вызываться при попытках вывести экземпляр класса или преобразовать его в строку.

Эти методы позволяют определить более удобочитаемый формат вывода ваших объектов. Строковое представление объектов по умолчанию не содержит полезной информации и имеет неудобочитаемый внешний вид.

In [48]:
class adder:
    def __init__(self, value=0):
        self.data = value        # инициализировать атрибут data
        
    def __add__(self, other):
        self.data += other       # прибавить другое значение
        
x = adder()
print(x)

In [49]:
x

In [50]:
class addrepr(adder):            # наследует __init__, __add__
    def __repr__(self):          # добавляет строковое представление
        return 'addrepr(%s)' % self.data

In [51]:
x = addrepr(2)  # вызовет __init__
x + 1           # вызовет __add__

In [52]:
x               # вызовет __repr__

addrepr(3)

In [53]:
print(x)        # вызовет __repr__

addrepr(3)


In [54]:
str(x), repr(x) # вызовет __repr__

('addrepr(3)', 'addrepr(3)')

Почему имеется два метода?

* Встроенные функции `print` и `str` (а также ее внутренний эквивалент, который используется функцией `print`) сначала пытаются использовать метод `__str__`. Вообще этот метод должен возвращать строковое представление объекта в удобном для пользователя виде.


* Во всех остальных случаях используется метод `__repr__`: функцией автоматического вывода в интерактивной оболочке, функцией `repr`, при выводе вложенных объектов, а также функциями `print` и `str`, когда в классе отсутствует метод `__str__`. Вообще этот метод должен возвращать строку, которая могла бы использоваться как программный код для воссоздания объекта или содержать информацию, полезную для разработчиков.

>Проще говоря, **метод `__repr__` используется везде, за исключением функций `print` и `str`, если определен метод `__str__`**.

>Однако, **если метод `__str__` отсутствует, операции вывода будут использовать метод `__repr__`**, но не наоборот – в остальных случаях, например, функцией автоматического вывода в интерактивной оболочке всегда используется только метод `__repr__`

Вследствие этого, если вам необходимо обеспечить единое отображение во всех контекстах, лучше использовать метод `__repr__`.

Однако, определив оба метода, вы обеспечите поддержку вывода в  различных контекстах. Например, перед конечным пользователем объект будет отображаться с помощью метода `__str__`, а перед программистом будет выводиться информация более низкого уровня с помощью метода `__repr__`:

In [55]:
class addboth(adder):
    def __str__(self):
        return '[Value: %s]' % self.data  # удобочитаемая строка
    
    def __repr__(self):
        return 'addboth(%s)' % self.data  # строка программного кода

In [56]:
x = addboth(4)
x + 1
x         # вызовет __repr__

addboth(5)

In [57]:
print(x)  # вызовет __str__

[Value: 5]


In [58]:
str(x), repr(x)

('[Value: 5]', 'addboth(5)')

>**Оба метода должны возвращать строки**, возвращаемые значения других типов не преобразуются в строки и вызывают ошибку

В зависимости от логики преобразования в строковое представление, реализованной в контейнерном объекте, **операция вывода может вызывать метод `__str__` только для объектов верхнего уровня** - **вложенные объекты по-прежнему могут выводиться с применением их методов `__repr__` или метода по умолчанию**

In [59]:
class Printer:
    def __init__(self, val):
        self.val = val
        
    def __str__(self):        # исп-ся для вывода самого экземпляра
        return str(self.val)  # преобразует результат в строку

In [60]:
objs = [Printer(2), Printer(3)]

for x in objs:
    print(x)

2
3


In [61]:
print(objs)

[<__main__.Printer object at 0x7f7f70988f28>, <__main__.Printer object at 0x7f7f70988da0>]


In [62]:
objs

[<__main__.Printer at 0x7f7f70988f28>, <__main__.Printer at 0x7f7f70988da0>]

Чтобы обеспечить вызов адаптированной версии метода во всех случаях, независимо от реализации контейнера, реализуйте метод `__repr__`, а не `__str__` – первый из них вызывается во всех случаях, где последний не может быть применен:

In [63]:
class Printer:
    def __init__(self, val):
        self.val = val
        
    def __repr__(self):   # ___repr__ исп-ся print, если нет __str__
        return str(self.val)  # __repr__ исп-ся инт. оболочкой и для
                              # вывода вложенных объектов

In [64]:
objs = [Printer(2), Printer(3)]

for x in objs:
    print(x)

2
3


In [65]:
print(objs)

[2, 3]


In [66]:
objs

[2, 3]

## Правостороннее сложение и операция приращения: `__radd__` и `__iadd__`

>C технической точки зрения метод `__add__`, который использовался в примерах выше, не поддерживает использование объектов экземпляров справа от оператора `+`. **Чтобы реализовать поддержку таких выражений и  тем самым обеспечить допустимость перестановки операндов, необходимо реализовать метод `__radd__`**.

**Интерпретатор вызывает метод `__radd__`, только когда экземпляр
вашего класса появляется справа от оператора `+`, а  объект слева не является экземпляром вашего класса**. Во всех других случаях, когда объект появляется слева, вызывается метод `__add__`:

In [67]:
class Commuter:
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        print('add', self.val, other)
        return self.val + other
    
    def __radd__(self, other):
        print('radd', self.val, other)
        return other + self.val

In [68]:
x = Commuter(88)
y = Commuter(99)
x + 1  # __add__: экземпляр + не_экземпляр

add 88 1


89

In [69]:
1 + y  # __radd__: не_экземпляр + экземпляр

radd 99 1


100

In [70]:
x + y  # __add__: экземпляр + экземпляр

add 88 <__main__.Commuter object at 0x7f7f70988080>
radd 99 88


187

На практике, когда требуется распространить тип класса на результат, реализация может оказаться сложнее: может оказаться необходимым выполнить проверку типа, чтобы убедиться в  безопасности операции преобразовании и избежать вложенности.

Так, если в следующем примере не выполнять проверку типа с  помощью функции `isinstance`, дело может закончиться тем, что мы получим экземпляр класса `Commuter`, значением атрибута `val` которого является другой экземпляр класса `Commuter`, – при сложении двух экземпляров,
когда метод `__add__` вызывает метод `__radd__`:

In [71]:
class Commuter:
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        if isinstance(other, Commuter):
            other = other.val
        return Commuter(self.val + other)
    
    def __radd__(self, other):
        return Commuter(other + self.val)
    
    def __str__(self):
        return '<Commuter: %s>' % self.val

In [72]:
x = Commuter(88)
y = Commuter(99)
print(x + 10)      # Результат – другой экземпляр класса Commuter

<Commuter: 98>


In [73]:
print(10 + y)

<Commuter: 109>


In [74]:
z = x + y    # Нет вложения: не происходит рекурсивный вызов __radd__
print(z)

<Commuter: 187>


In [75]:
print(z + 10)

<Commuter: 197>


In [76]:
print(z + z)

<Commuter: 374>


### Комбинированная операция сложения

Чтобы обеспечить поддержку комбинированной операции сложения `+=`, увеличивающей значение экземпляра, необходимо реализовать метод `__iadd__` или `__add__`. Последний из них используется в случае отсутствия первого. 

Фактически класс `Commuter`, представленный в предыдущем разделе, уже поддерживает операцию `+=`, однако с помощью метода `__iadd__` можно реализовать более эффективную операцию изменения самого экземпляра

In [77]:
class Number:
    def __init__(self, val):
        self.val = val
        
    def __iadd__(self, other): # __iadd__ явно реализует операцию x += y
        self.val += other      # Обычно возвращает self
        return self

In [78]:
x = Number(5)
x += 1

In [79]:
x += 1

In [80]:
x.val

7

In [81]:
class Number:
    def __init__(self, val):
        self.val = val
    
    def __add__(self, other): # __add__ как крайнее средство: x=(x + y)
        return Number(self.val + other) # распространяет тип класса

In [82]:
x = Number(5)
x += 1

In [83]:
x += 1

In [84]:
x.val

7

## Операция вызова: `__call__`

**Метод `__call__` вызывается при обращении к экземпляру как к функции** и поддерживает все схемы передачи аргументов - все, что передается экземпляру, передается этому методу наряду с аргументом `self`, в котором передается сам экземпляр.

In [85]:
class Callee:
    def __call__(self, *pargs, **kargs): # реализует вызов экземпляра
        print('Called:', pargs, kargs)   # принимает любые аргументы

In [86]:
C = Callee()
C(1, 2, 3)

Called: (1, 2, 3) {}


In [87]:
C(1, 2, 3, x=4, y=5)

Called: (1, 2, 3) {'x': 4, 'y': 5}


Реализация операции вызова позволяет экземплярам классов имитировать поведение функций, а также сохранять информацию о состоянии между вызовами.

In [89]:
class Prod:
    def __init__(self, value):  # принимает единственный аргумент
        self.value = value
    
    def __call__(self, other):
        return self.value * other

In [90]:
x = Prod(2)  # запоминает 2 в своей области видимости
x(3)         # 3 (передано) * 2 (сохраненное значение)

6

>Метод `__call__` может оказаться удобнее **при взаимодействии с прикладными интерфейсами, где ожидается функция**, - это позволяет создавать объекты, совместимые с интерфейсами, ожидающими получить функцию, которые к тому же способны сохранять информацию о своем состоянии между вызовами.

## Функциональные интерфейсы и программный код обратного вызова

Инструментальный набор для создания графического интерфейса `tkinter` позволяет регистрировать функции как **обработчики событий (они же - функции обратного вызова)**. Когда возникают какие-либо события, `tkinter` вызывает зарегистрированные объекты.

Если необходимо реализовать обработчик событий, способный сохранять свое состояние между вызовами, вы можете **либо зарегистрировать связанный метод класса, либо экземпляр класса, который с помощью метода `__call__` обеспечивает совместимость с функциональным интерфейсом**.

Например:

In [91]:
class Callback:
    def __init__(self, color):  # функция + информация о состоянии
        self.color = color
        
    def __call__(self):         # поддерживает вызовы без аргументов
        print('turn', self.color)

Теперь можно зарегистрировать экземпляры этого класса в контексте графического интерфейса, как обработчики событий для кнопок, несмотря на то, что реализация графического интерфейса предполагает вызывать обработчики событий как обычные функции без аргументов:

```
cb1 = Callback('blue')  # "запомнить" голубой цвет
cb2 = Callback('green')

B1 = Button(command=cb1)  # зарегистрировать обработчик
B2 = Button(command=cb2)  # зарегистрировать обработчик
```

Когда позднее кнопка будет нажата, объект экземпляра будет вызван как простая функция, а поскольку он сохраняет информацию о состоянии в атрибутах экземпляра, он помнит, что необходимо сделать

Еще 2 способа, которые используются для сохранения информации о состояниях в функциях обратного вызова.

**1. lambda-функция с аргументами, имеющими значения по умолчанию**
```
cb3 = (lambda color='red': 'turn' + color)
```

**2. Связанные методы класса**
Объект связанного метода - это объект, который запоминает экземпляр `self` и ссылку на функцию, так что потом можно вызывать простую функцию без использования экземпляра

In [92]:
class Callback:
    def __init__(self, color):  # класс с информацией о состоянии
        self.color = color
        
    def changeColor(self):      # обычный метод
        print('turn', self.color)

```
cb1 = Callback('blue')
cb2 = Callback('green')

B1 = Button(command=cb1.changeColor)  # ссылка, не вызов
B2 = Button(command=cb2.changeColor)  # запом-ся ф-ия + self
```

Когда позднее кнопка будет нажата, имитируется поведение графического интерфейса и вызывается метод `changeColor`, который обработает информацию о состоянии объекта:

```
obj = Callback('blue')
cb = obj.changeColor  # регистрация обработчика событий
cb()                  # по событию выведет 'blue'
```

Этот прием более простой, но менее универсальный, чем перегрузка операции вызова с помощью метода `__call__`

## Сравнение: `__lt__`, `__gt__` и другие

Классы могут определять методы, реализующие все шесть операций сравнения: `<`, `>`, `<=`, `>=`, `==` и `!=`. Обычно они просты в реализации, но нужно иметь в виду следующее:

* В отличие от методов `__add__`/`__radd__`, обсуждавшихся выше, **методы сравнения не имеют правосторонних версий**. Вместо этого, когда операцию сравнения поддерживает только один операнд, используются **зеркальные методы сравнивания** (например, методы `__lt__` и `__gt__` являются зеркальными по отношению друг к другу).


* **Среди операторов сравнения нет неявных взаимоотношений**. Суть в  том, что истинность операции `==` не предполагает ложность операции `!=`, например, чтобы гарантировать корректное поведение обоих операторов, требуется реализовать оба метода, `__eq__` и `__ne__`.

Пример:

In [93]:
class C:
    data = 'spam'
    
    def __gt__(self, other):
        return self.data > other
    
    def __lt__(self, other):
        return self.data < other

In [94]:
X = C()
print(X > 'ham')  # вызовет __gt__

True


In [95]:
print(X < 'ham')

False


## Проверка логического значения: `__bool__` и `__len__`

Классы могут определять методы, выражающие логическую природу их экземпляров, - в логическом контексте интерпретатор сначала пытается напрямую получить логическое значение с помощью метода `__bool__` и только потом, если этот метод не реализован, пытается вызвать метод `__len__`, чтобы выяснить истинность объекта, исходя из его длины.

Обычно `__bool__` возвращает логическое значение, исходя из значений атрибутов объекта или другой информации:

In [96]:
class Truth:
    def __bool__(self):
        return True
    
X = Truth()
if X:
    print('yes!')

yes!


In [97]:
class Truth:
    def __bool__(self):
        return False
    
X = Truth()
bool(X)

False

Если метод отсутствует, интерпретатор пытается определить длину объекта, поскольку непустой объект интерпретируется как истинный:

In [98]:
class Truth:
    def __len__(self):
        return 0
    
X = Truth()
if not X:
    print('no!')

no!


**Если реализованы оба метода, предпочтение отдается методу `__bool__`**, потому что он является более специализированным

In [99]:
class Truth:
    def __bool__(self):
        return True
    
    def __len__(self):
        return 0
    
X = Truth()
if X:
    print('yes!')

yes!


>**Если ни один из методов не определен, объект просто считается истинным**

In [100]:
class Truth:
    pass

X = Truth()
bool(X)

True

## Уничтожение объектов: `__del__`

Конструктор `__init__` вызывается во время создания экземпляра. Противоположный ему **метод `__del__` вызывается автоматически, когда освобождается память, занятая объектом (то есть во время "сборки мусора")**.

In [101]:
class Life:
    def __init__(self, name='unknown'):
        print('Hello', name)
        self.name = name
        
    def __del__(self):
        print('Goodbye', self.name)

In [102]:
brian = Life('Brian')

Hello Brian


In [103]:
brian = 'loretta'

Goodbye Brian


Здесь, когда переменной `brian` присваивается строка, теряется последняя ссылка на экземпляр класса `Life`, что приводит к вызову деструктора.

С одной стороны, интерпретатор автоматически освобождает память, занятую экземпляром, поэтому нет нужды выполнять очистку памяти в деструкторах.

С другой стороны, не всегда бывает возможным предсказать, когда произойдет уничтожение экземпляра, поэтому часто лучше выполнять завершающие действия в явно вызываемом методе (или в инструкции `try/finally` - в некоторых случаях в системных таблицах могут сохраняться ссылки на ваши объекты, что будет препятствовать вызову деструктора.